In [1]:
import qlib
from qlib.constant import REG_CN
provider_uri = "../../data/qlib_data/cn_data_ak"  # target_dir
qlib.init(provider_uri=provider_uri, region=REG_CN)

[17368:MainThread](2022-09-01 22:53:33,510) INFO - qlib.Initialization - [config.py:413] - default_conf: client.
[17368:MainThread](2022-09-01 22:53:34,247) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[17368:MainThread](2022-09-01 22:53:34,247) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': WindowsPath('D:/PycharmProjects/QlibPlatform/data/qlib_data/cn_data_ak')}


In [2]:
# 导入QlibDataLoader
from qlib.data.dataset.loader import QlibDataLoader

In [19]:
# 加载原始特征，比如收盘价、最高价
qdl = QlibDataLoader(config=(['$close', '$high'],['close', 'high'])) 
df = qdl.load(instruments=['sh600001'], start_time='20010101', end_time='20191231') # 可以通过freq参数设置周期，默认freq='day'

In [24]:
df

,,close,high
datetime,instrument,,
2001-01-02,sh600001,10.08,10.09
2001-01-03,sh600001,10.06,10.16
2001-01-04,sh600001,10.04,10.16
2001-01-05,sh600001,10.25,10.32
2001-01-08,sh600001,10.50,10.58
...,...,...,...
2009-12-09,sh600001,13.78,14.08
2009-12-10,sh600001,13.91,14.10
2009-12-11,sh600001,13.64,14.17


In [4]:
sfrom qlib.contrib.data.handler import Alpha158
from qlib.data.dataset import TSDatasetH
from qlib.contrib.model.pytorch_alstm_ts import ALSTM

# 配置数据
train_period = ("2015-01-01", "2019-12-31")
valid_period = ("2020-01-01", "2020-12-31")
test_period = ("2021-01-01", "2022-03-01")

dh = Alpha158(instruments='csi300', 
              start_time=train_period[0], 
              end_time=test_period[1],
              infer_processors={}
              
             )
ds = TSDatasetH(handler=dh,
                step_len=40, # 时间步数
                segments={"train": train_period, 
                          "valid": valid_period, 
                          "test": test_period})

Please install necessary libs for CatBoostModel.
ModuleNotFoundError. XGBModel is skipped(optional: maybe installing xgboost can fix it).


[17368:MainThread](2022-09-01 22:54:19,207) INFO - qlib.timer - [log.py:117] - Time cost: 40.076s | Loading data Done
[17368:MainThread](2022-09-01 22:54:19,548) INFO - qlib.timer - [log.py:117] - Time cost: 0.278s | DropnaLabel Done
C:\Users\lyg\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
[17368:MainThread](2022-09-01 22:54:21,233) INFO - qlib.timer - [log.py:117] - Time cost: 1.684s | CSZScoreNorm Done
[17368:MainThread](2022-09-01 22:54:21,253) INFO - qlib.timer - [log.py:117] - Time cost: 2.046s | fit & process data Done
[17368:MainThread](2022-09-01 22:54:21,253) INFO - qlib.timer - [log.py:117] - Time cost: 42.123s | Init data Done


In [5]:
dh.fetch().isna().sum()

KMID       14725
KLEN       14725
KMID2      14725
KUP        14725
KUP2       14725
           ...  
VSUMD10    11195
VSUMD20     9169
VSUMD30     7693
VSUMD60     4547
LABEL0     15395
Length: 159, dtype: int64

In [15]:
dh.fetch()

KMID      KLEN     KMID2       KUP      KUP2  \
datetime   instrument                                                     
2015-01-05 SH600000    0.010582  0.038240  0.276729  0.009981  0.261007   
           SH600008    0.021103  0.071750  0.294117  0.028700  0.400000   
           SH600009    0.031786  0.048738  0.652175  0.016952  0.347825   
           SH600010    0.054342  0.067443  0.805754  0.001941  0.028778   
           SH600011    0.038429  0.052946  0.725806  0.008540  0.161291   
...                         ...       ...       ...       ...       ...   
2022-03-01 SZ300782   -0.006568  0.014570 -0.450778  0.003574  0.245314   
           SZ300866   -0.012725  0.018978 -0.670521  0.001426  0.075143   
           SZ300888    0.002503  0.009881  0.253326  0.002108  0.213338   
           SZ300896    0.001231  0.019391  0.063493  0.009131  0.470900   
           SZ300999    0.020004  0.026734  0.748251  0.000561  0.020978   

                           KLOW     KLOW2      KSFT     KSFT2     OPEN0  ...  \
datetime   instrument                                                    ...   
2015-01-05 SH600000    0.017677  0.462264  0.018278  0.477986  0.989529  ...   
           SH600008    0.021947  0.305883  0.014350  0.200001  0.979333  ...   
           SH600009    0.000000  0.000000  0.014833  0.304350  0.969193  ...   
           SH600010    0.011160  0.165469  0.063561  0.942445  0.948458  ...   
           SH600011    0.005978  0.112903  0.035867  0.677418  0.962993  ...   
...                         ...       ...       ...       ...       ...  ...   
2022-03-01 SZ300782    0.004428  0.303908 -0.005714 -0.392185  1.006611  ...   
           SZ300866    0.004827  0.254336 -0.009324 -0.491328  1.012889  ...   
           SZ300888    0.005270  0.533335  0.005665  0.573324  0.997503  ...   
           SZ300896    0.009029  0.465607  0.001129  0.058200  0.998770  ...   
           SZ300999    0.006169  0.230770  0.025612  0.958044  0.980389  ...   

                        VSUMN10   VSUMN20   VSUMN30   VSUMN60    VSUMD5  \
datetime   instrument                                                     
2015-01-05 SH600000    0.493269  0.534596  0.460477  0.473383  0.137414   
           SH600008    0.637252  0.537391  0.465414  0.487741 -0.068779   
           SH600009    0.400644  0.477602  0.462575  0.475181  0.668636   
           SH600010    0.615322  0.558431  0.467880  0.498684  0.451477   
           SH600011    0.431567  0.464726  0.430197  0.448191  0.345714   
...                         ...       ...       ...       ...       ...   
2022-03-01 SZ300782    0.542146  0.506324  0.501819  0.532878 -0.132422   
           SZ300866    0.448181  0.471702  0.513416  0.560920  0.219701   
           SZ300888    0.533962  0.578113  0.714335  0.542723 -0.109877   
           SZ300896    0.647234  0.499206  0.508283  0.499548  0.008859   
           SZ300999    0.429111  0.502528  0.494542  0.498051  0.135325   

                        VSUMD10   VSUMD20   VSUMD30   VSUMD60    LABEL0  
datetime   instrument                                                    
2015-01-05 SH600000    0.013462 -0.069193  0.079047  0.053234 -0.017552  
           SH600008   -0.274505 -0.074782  0.069172  0.024518 -0.035751  
           SH600009    0.198712  0.044795  0.074850  0.049638  0.003807  
           SH600010   -0.230643 -0.116862  0.064241  0.002633  0.015559  
           SH600011    0.136865  0.070547  0.139607  0.103618 -0.000841  
...                         ...       ...       ...       ...       ...  
2022-03-01 SZ300782   -0.084293 -0.012648 -0.003638 -0.065756 -0.029538  
           SZ300866    0.103638  0.056596 -0.026831 -0.121840 -0.026265  
           SZ300888   -0.067925 -0.156225 -0.428670 -0.085446  0.015697  
           SZ300896   -0.294469  0.001588 -0.016565  0.000903 -0.023241  
           SZ300999    0.141777 -0.005055  0.010915  0.003898 -0.014997  

[521801 rows x 159 columns]